# GIS analysis with geopandas - CO weed sales

Let's take a look at one year of [county-level cannabis sales data in Colorado](https://www.colorado.gov/pacific/revenue/colorado-marijuana-sales-reports). We shall plot it out on a map.

**Our goal**: To map per-capita retail cannabis sales in Colorado by county in 2017.

**Our data**: A CSV lives here: `../../data/colo-cannabis-sales.csv`.

A few things to note about this data:
- Every row in our data is the sum of one month of sales for one category of cannabis ("retail" or "medical") for one county
- Not every county in Colorado has retail pot shops
- To maintain taxpayer privacy, the state releases aggregate sales data _only_ for counties with at least three dispensaries, and then only if none represent more than 80 percent of total sales, according to the Colorado Department of Revenue. Totals for counties that don't meet these criteria are represented in the data as 'NR'

Other data we will use as we explore this topic:
- `../../data/colo-county-pop.csv`: A CSV with Colorado county [FIPS](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standards) codes, county names and 2016 population estimates.
- `../../data/co-counties.shp`: A Census Bureau [shapefile](https://en.wikipedia.org/wiki/Shapefile) of Colorado counties.

### Import dependencies

We'll need pandas and geopandas.

We'll also set up a ["magic function"](https://stackoverflow.com/a/43028034) to plot our map inline.

In [ ]:
# import pandas and geopandas and call our magic function for matplotlib


### Read in the weed data

In [ ]:
# read in the cannabis sales data csv


In [ ]:
# check the output with `head()`


### Read in the county data

Again, we're going to use the `dtype` argument to make sure that the `fips` column comes in as a string.

In [ ]:
# read in the county population data CSV


In [ ]:
# check the output with `head()`


### Filter for retail sales

First, use `unique()` to see what values are in that column. Then filter for the one we want.

In [ ]:
# use `unique()` to figure out what types of weed we're dealing with


In [ ]:
# filter for retail


In [ ]:
# check the output with `head()`


### Filter for 2017 data

In [ ]:
# filter ~that~ dataframe to just get data for 2017


### Group weed data by county for annual total

Every row in our data is one month of sales data -- we want to get the annual total. So we'll use the `groupby()` and `sum()` methods to calculate that.

The object we end up with isn't _quite_ what we need to join this data with our other dataframes, though, so we'll also [reset the index](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.reset_index.html) to get back to something we can work with.

In [ ]:
# select county and amount columns, `groupby()` county, sum values, reset index


In [ ]:
# check the output with `head()`


### Join the grouped weed data with the county data

Now we'll smash together the weed data and the county data, and to do that, we're going to use the pandas [`merge()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) method. We'll pass it four arguments:
- `grouped`, the "left" dataframe we're joining
- `county_df`, the "right" dataframe we're joining
- `left_on='county'`, which tells us which column in the _left_ dataframe to join on
- `right_on='county_name'`, which tells us which column in the _right_ dataframe to join on

In [ ]:
# join that data with the county dataframe


In [ ]:
# check the output with `head()`


### Calculate per-capita weed sales

The math is pretty simple -- divide the amount into the population -- so adding a calculated column is also pretty simple.

Then we'll sort by our new column to see which county rises to the top.

In [ ]:
# calculate per-capita weed sales in a new column


In [ ]:
# sort descending and check the output with `head()`


### Let's start prepping that map

A table is cool and all, but sometimes patterns emerge from looking at a visualization. Let's plot our data as a [choropleth map](https://en.wikipedia.org/wiki/Choropleth_map).

First, let's just grab the data that we need.

In [ ]:
# select only the columns of data we're interested in


### Read in the shapefile data

The geopandas [`read_file()`](http://geopandas.org/io.html#reading-spatial-data) method is our horse.

In [ ]:
# read in the colorado counties shapefile


In [ ]:
# check the output with `head()`


### Merge the GIS file with our grouped dataframe

We're going to use `merge()` again. This time, though, we're going to hand it _five_ arguments:
- `county_gis[['GEOID10', 'NAME10', 'geometry']]`, the "left" dataframe to join, in this case a slimmed-down version of the shapefile we just read in
- `data_to_map`, the "right" dataframe to join
- `how='left'`, which specifies what _kind_ of join we're doing -- in this case, we want to keep all of the counties in our geodataframe (on the left), even if they don't have any associated data, and the [default](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) for this argument is "inner"
- `left_on='GEOID10'`, which specifies the column name in the "left" dataframe to join on
- `right_on='county_code'`, which specifies the column name in the "right" dataframe to join on

In [ ]:
# merge the county GIS data with the data to map


In [ ]:
# check the output with `head()`


### (Finally!) draw the map

We're going to hand the [`plot()`](http://geopandas.org/mapping.html) method four arguments: 
- `column='per_capita'`, which [specifies](http://geopandas.org/mapping.html#choropleth-maps) the column of data to use to calculate the color of each county
- `edgecolor='black'`, which specifies what color to outline the counties in
- `cmap='YlGn'`, which specifies the [colormap](https://matplotlib.org/users/colormaps.html) to use
- `figsize=(13,7)`, which sets the size of the output

In [ ]:
# plot the results on a map


Spot the outlier ...